In [11]:
import cv2 as cv
import numpy as np
from skimage import data
from skimage.filters import threshold_local, median
from skimage import exposure
from skimage import img_as_ubyte
from skimage.morphology import erosion, dilation, opening, closing, white_tophat
from skimage.morphology import disk
from os import listdir


basepath_training = 'DRIVE/training/images/'
basepath_groundtruth = 'DRIVE/training/1st_manual/'

training_files = listdir(basepath_training)
groundtruth_files = listdir(basepath_groundtruth)
files_count = len(training_files)

sensitivity = 0
specificity = 0
accuracy = 0
selem = disk(1.3)
it = 1
for img_paths in zip(training_files, groundtruth_files):

    # segmentation of original image
    img = cv.imread(basepath_training + img_paths[0])
    b,g,r = cv.split(img)
    img_shape = g.shape[:2]

    th2 = exposure.equalize_adapthist(g, clip_limit=0.016)
    med = median(th2)
    block_size = 13
    bina = th2 > threshold_local(th2, block_size, method='mean', offset=0.042)
    bina = np.invert(bina)
    img = img_as_ubyte(bina)
    result1 = opening(img, selem)
    selem = disk(1.4)
    result = opening(result1, selem)
    
    # ground truth
    cap = cv.VideoCapture(basepath_groundtruth + img_paths[1])
    ret, frame = cap.read()

    tp = 0; tn = 0; fp = 0; fn = 0
    for i in range(img_shape[0]):
        for j in range(img_shape[1]):
            px = result[i][j]
            if px == 0:
                if frame[i,j,1] == 0:
                    tn = tn + 1
                elif frame[i,j,1] == 255:
                    fn = fn + 1
            elif px == 255:
                if frame[i,j,1] == 0:
                    fp = fp + 1
                elif frame[i,j,1] == 255:
                    tp = tp + 1
                
    sen = tp / (tp+fn) * 100
    spec = tn / (tn+fp) * 100
    acc = (tp + tn) / (tp + tn + fp + fn) * 100 


    sensitivity = sensitivity + sen
    specificity = specificity + spec
    accuracy = accuracy + acc
    print('Progress: ' + (it * '|') + ((files_count-it) * '.') + ' %.2f' % (it / files_count * 100) + '%')
    it = it + 1

print('sen: %d, spec: %d, acc: %d' % (sensitivity / files_count, \
            specificity / files_count, \
            accuracy / files_count))



C:\ProgramData\Anaconda3\lib\site-packages\skimage\util\dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\ProgramData\Anaconda3\lib\site-packages\skimage\util\arraycrop.py:177: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  cropped = ar[slices]


Progress: |................... 5.00%
Progress: ||.................. 10.00%
Progress: |||................. 15.00%
Progress: ||||................ 20.00%
Progress: |||||............... 25.00%
Progress: ||||||.............. 30.00%
Progress: |||||||............. 35.00%
Progress: ||||||||............ 40.00%
Progress: |||||||||........... 45.00%
Progress: ||||||||||.......... 50.00%
Progress: |||||||||||......... 55.00%
Progress: ||||||||||||........ 60.00%
Progress: |||||||||||||....... 65.00%
Progress: ||||||||||||||...... 70.00%
Progress: |||||||||||||||..... 75.00%
Progress: ||||||||||||||||.... 80.00%
Progress: |||||||||||||||||... 85.00%
Progress: ||||||||||||||||||.. 90.00%
Progress: |||||||||||||||||||. 95.00%
Progress: |||||||||||||||||||| 100.00%
sen: 66, spec: 94, acc: 92


In [ ]:
66 94 92
68 94 91